<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Burp_Fuzzing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Extending Burp Proxy__*

If you've ever tried hacking, you're a NIGRO.